



#################################################################################################################################################################################################################################################################################################################################################
# Collaboration and Competition
---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import os

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app", worker_id=303, no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [4]:
"""
for i in range(1, 2):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))
"""

"\nfor i in range(1, 2):                                      # play game for 5 episodes\n    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    \n    states = env_info.vector_observations                  # get the current state (for each agent)\n    scores = np.zeros(num_agents)                          # initialize the score (for each agent)\n    while True:\n        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)\n        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1\n        env_info = env.step(actions)[brain_name]           # send all actions to tne environment\n        next_states = env_info.vector_observations         # get next state (for each agent)\n        rewards = env_info.rewards                         # get reward (for each agent)\n        dones = env_info.local_done                        # see if episode finished\n        scores += env_info.rewards               

In [5]:
for i in zip(states,[1,2]):
    print(i)

(array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -6.65278625, -1.5       , -0.        ,  0.        ,
        6.83172083,  6.        , -0.        ,  0.        ]), 1)
(array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -6.4669857 , -1.5       ,  0.        ,  0.        ,
       -6.83172083,  6.        ,  0.        ,  0.        ]), 2)


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
import torch
import numpy as np
from ma_ddpg import MADDPG

model_dir = 'saved_models/'
model_name = 'multi-agents_' + str(brain_name) + '_' + str(num_agents) + '_agents.pt'

os.makedirs(model_dir, exist_ok=True)

episode_max = 15000
rollout_len = 1000

ma_ddpg = MADDPG(state_size, action_size, num_agents, seed=1234)

In [8]:
def saveTrainedModel(agent, path):
    state_dicts = {}
    for i in range(num_agents):
        state_dicts['model_agent' + str(i)] = {'actor_local': ma_ddpg.maddpg_agent[i].actor_local.state_dict(), 
                                               'critic_local': ma_ddpg.maddpg_agent[i].critic_local.state_dict(), 
                                               'actor_target': ma_ddpg.maddpg_agent[i].actor_target.state_dict(), 
                                               'critic_target': ma_ddpg.maddpg_agent[i].critic_target.state_dict()}
    torch.save(state_dicts, path)

In [ ]:
agents_scores_history = [[] for _ in range(num_agents)]
max_episodic_scores_history = []
average_scores_history = []
t_history = []
e = 0
isSolved = False

while e < episode_max:
    
    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state np array :num_agents x state_size
    running_score = np.zeros(num_agents)                   # initialize the score (for each agent)
    
    t = 0
    
    while True:
        actions_list = ma_ddpg.acts(states)                # select an action (for each agent)
        actor_tensor = torch.stack(actions_list).cpu().detach()
        actions = np.clip(actor_tensor.numpy(), -1, 1)     # all actions between -1 and 1 #num_agents x action_size
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (# num_agents x state_size)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        
        ma_ddpg.step((states, actor_tensor, rewards, dones, next_states))
        
        running_score += env_info.rewards                  # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones) or t >= rollout_len:   
            break
            t_history.append[t]
            
        t += 1

    for i in range(num_agents):
        agents_scores_history[i].append(running_score[i])
        
    max_episodic_scores_history.append(np.max(running_score))
    
    avg_100_score = np.mean(max_episodic_scores_history[-100:])
    average_scores_history.append(avg_100_score)

    if ma_ddpg.is_training:
        if (e+1)%10==0:
            print("e: {}  score(s): {:.3f} {:.3f}  Avg score: {:.3f}  "
                  "Actor(g): {:.2e}  Critic(l): {:.2e}  n: {:.2f}  "
                  "s: {}".format(e+1, np.mean(agents_scores_history[0][-100:]), 
                                     np.mean(agents_scores_history[1][-100:]), 
                                     np.mean(avg_100_score),
                                     np.mean(ma_ddpg.ag_history), 
                                     np.mean(ma_ddpg.cl_history),
                                     np.mean(ma_ddpg.noise_history), ma_ddpg.t_step))
            
        if not isSolved and np.mean(avg_100_score) >= 0.5:
            print('\033[32m env solved at episode {}!'.format(e-100))
            isSolved = True
                   
        e+=1
    else:
        print('\rFetching experiences... {} '.format(ma_ddpg.t_step), end="")

Fetching experiences... 4989 Prefetch experience completed. Training starts! 
Number of Agents:  2
Device:  cpu
CRITIC_ACT_FORM:  3
BATCH_SIZE:  128
LR:  0.001
REWARD_SCALE:  10.0
LEARN_EVERY:  1
LEARN_LOOP:  4
UNITS_ACTOR:  (256, 128)
UNITS_CRITIC:  (256, 128)
NOISE_WGT_DECAY:  0.9993
NOISE_DECAY_EVERY:  5
NOISE_WGT_MIN:  0.2
USE_PER:  True
USE_BATCHNORM:  False
e: 10  score(s): 0.000 0.002  Avg score: 0.011  Actor(g): 2.49e-01  Critic(l): 6.17e+02  n: 1.44  s: 5171
e: 20  score(s): -0.001 0.004  Avg score: 0.012  Actor(g): 4.62e-01  Critic(l): 3.80e+02  n: 1.25  s: 5345
e: 30  score(s): 0.000 0.001  Avg score: 0.010  Actor(g): 6.03e-01  Critic(l): 2.70e+02  n: 1.14  s: 5504
e: 40  score(s): 0.003 0.000  Avg score: 0.012  Actor(g): 7.58e-01  Critic(l): 1.84e+02  n: 1.09  s: 5698
e: 50  score(s): 0.002 -0.001  Avg score: 0.010  Actor(g): 9.01e-01  Critic(l): 1.41e+02  n: 1.59  s: 5840
e: 60  score(s): 0.000 -0.000  Avg score: 0.009  Actor(g): 9.59e-01  Critic(l): 8.29e+01  n: 1.23  s: 

e: 740  score(s): 0.010 0.002  Avg score: 0.021  Actor(g): 1.60e+00  Critic(l): 2.23e+01  n: 0.25  s: 17186
e: 750  score(s): 0.010 0.002  Avg score: 0.021  Actor(g): 1.46e+00  Critic(l): 1.93e+01  n: 0.21  s: 17369
e: 760  score(s): 0.007 0.002  Avg score: 0.018  Actor(g): 1.41e+00  Critic(l): 2.30e+01  n: 0.30  s: 17543
e: 770  score(s): 0.005 0.003  Avg score: 0.017  Actor(g): 1.36e+00  Critic(l): 2.32e+01  n: 0.24  s: 17727
e: 780  score(s): 0.003 0.004  Avg score: 0.016  Actor(g): 1.57e+00  Critic(l): 2.45e+01  n: 0.20  s: 17903
e: 790  score(s): 0.001 0.006  Avg score: 0.015  Actor(g): 1.60e+00  Critic(l): 2.68e+01  n: 0.21  s: 18078
e: 800  score(s): 0.002 0.007  Avg score: 0.016  Actor(g): 1.56e+00  Critic(l): 2.59e+01  n: 0.19  s: 18238
e: 810  score(s): 0.000 0.006  Avg score: 0.015  Actor(g): 1.55e+00  Critic(l): 2.56e+01  n: 0.24  s: 18383
e: 820  score(s): -0.000 0.005  Avg score: 0.014  Actor(g): 1.60e+00  Critic(l): 2.51e+01  n: 0.20  s: 18525
e: 830  score(s): -0.000 0.

e: 1490  score(s): 0.003 -0.005  Avg score: 0.008  Actor(g): 1.54e+00  Critic(l): 1.96e+01  n: 0.06  s: 28265
e: 1500  score(s): 0.005 -0.005  Avg score: 0.010  Actor(g): 1.57e+00  Critic(l): 1.74e+01  n: 0.06  s: 28443
e: 1510  score(s): 0.007 -0.005  Avg score: 0.012  Actor(g): 1.55e+00  Critic(l): 1.75e+01  n: 0.07  s: 28640
e: 1520  score(s): 0.009 -0.005  Avg score: 0.014  Actor(g): 1.59e+00  Critic(l): 1.71e+01  n: 0.06  s: 28835
e: 1530  score(s): 0.010 -0.005  Avg score: 0.015  Actor(g): 1.53e+00  Critic(l): 1.76e+01  n: 0.06  s: 29014
e: 1540  score(s): 0.011 -0.006  Avg score: 0.015  Actor(g): 1.48e+00  Critic(l): 1.71e+01  n: 0.07  s: 29216
e: 1550  score(s): 0.012 -0.006  Avg score: 0.015  Actor(g): 1.52e+00  Critic(l): 1.69e+01  n: 0.06  s: 29364
e: 1560  score(s): 0.012 -0.007  Avg score: 0.015  Actor(g): 1.51e+00  Critic(l): 1.60e+01  n: 0.06  s: 29522
e: 1570  score(s): 0.012 -0.007  Avg score: 0.015  Actor(g): 1.44e+00  Critic(l): 1.52e+01  n: 0.05  s: 29697
e: 1580  s

e: 2240  score(s): 0.025 -0.008  Avg score: 0.027  Actor(g): 1.71e+00  Critic(l): 1.56e+01  n: 0.06  s: 42988
e: 2250  score(s): 0.026 -0.008  Avg score: 0.028  Actor(g): 1.70e+00  Critic(l): 1.69e+01  n: 0.06  s: 43206
e: 2260  score(s): 0.026 -0.008  Avg score: 0.028  Actor(g): 1.79e+00  Critic(l): 1.63e+01  n: 0.06  s: 43376
e: 2270  score(s): 0.025 -0.008  Avg score: 0.026  Actor(g): 1.67e+00  Critic(l): 1.69e+01  n: 0.07  s: 43550
e: 2280  score(s): 0.022 -0.008  Avg score: 0.024  Actor(g): 1.74e+00  Critic(l): 1.70e+01  n: 0.05  s: 43725
e: 2290  score(s): 0.027 -0.008  Avg score: 0.029  Actor(g): 1.79e+00  Critic(l): 1.77e+01  n: 0.05  s: 43946
e: 2300  score(s): 0.028 -0.008  Avg score: 0.030  Actor(g): 1.75e+00  Critic(l): 1.70e+01  n: 0.07  s: 44163
e: 2310  score(s): 0.029 -0.008  Avg score: 0.031  Actor(g): 1.71e+00  Critic(l): 1.85e+01  n: 0.06  s: 44374
e: 2320  score(s): 0.029 -0.008  Avg score: 0.030  Actor(g): 1.53e+00  Critic(l): 1.36e+01  n: 0.06  s: 44565
e: 2330  s

In [ ]:
# wrong wrong same id entry mixed detached with activation, longer decay time
saveTrainedModel(ma_ddpg, model_dir + model_name)

In [ ]:
# plot the scores
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(max_episodic_scores_history)), max_episodic_scores_history)
plt.plot(np.arange(len(average_scores_history)), average_scores_history) # plotting t, c separately 
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
len(average_scores_history)